# correlation between DASB & MADAM

In [8]:
import numpy as np
import pandas as pd
from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [9]:
from neuromaps.datasets import available_annotations

for annotation in available_annotations():

    print(annotation)

('abagen', 'genepc1', 'fsaverage', '10k')
('aghourian2017', 'feobv', 'MNI152', '1mm')
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')
('bedard2019', 'feobv', 'MNI152', '1mm')
('beliveau2017', 'az10419369', 'MNI152', '1mm')
('beliveau2017', 'az10419369', 'fsaverage', '164k')
('beliveau2017', 'cimbi36', 'MNI152', '1mm')
('beliveau2017', 'cimbi36', 'fsaverage', '164k')
('beliveau2017', 'cumi101', 'MNI152', '1mm')
('beliveau2017', 'cumi101', 'fsaverage', '164k')
('beliveau2017', 'dasb', 'MNI152', '1mm')
('beliveau2017', 'dasb', 'fsaverage', '164k')
('beliveau2017', 'sb207145', 'MNI152', '1mm')
('beliveau2017', 'sb207145', 'fsaverage', '164k')
('ding2010', 'mrb', 'MNI152', '1mm')
('dubois2015', 'abp688', 'MNI152', '1mm')
('dukart2018', 'flumazenil', 'MNI152', '3mm')
('dukart2018', 'fpcit', 'MNI152', '3mm')
('fazio2016', 'madam', 'MNI152', '3mm')
('finnema2016', 'ucbj', 'MNI152', '1mm')
('gallezot2010', 'p943', 'MNI152', '1mm')
('gallezot2017', 'gsk189254', 'MNI152', '1mm')
('hcps1200', 'm

In [10]:
from neuromaps.images import annot_to_gifti
from neuromaps.parcellate import Parcellater

# define path
path = '/Users/laurituominen/Documents/Research/Reettis/neuromaps/'

# load in different parcellation files
dk_fsaverage_164k = (path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot',
                     path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
dk_mni = path + 'parcellations/atlas-desikankilliany_space-MNI_res-1mm.nii.gz'

# make sure label IDs are consecutive across hemispheres
dk_fsaverage_164k = annot_to_gifti(dk_fsaverage_164k)  # this does relabel_gift and also converts the annot file to gifti

# make the parcellaters for each space
parcellater_fs164k = Parcellater(dk_fsaverage_164k, 'fsaverage')
parcellater_mni = Parcellater(dk_mni, 'MNI152')

In [16]:
from neuromaps.datasets import fetch_annotation
dasb = fetch_annotation(source='beliveau2017', desc='dasb', space='fsaverage')
dasb_mni_v1 = fetch_annotation(source='beliveau2017', desc='dasb', space='MNI152')
dasb_mni_v2 = fetch_annotation(source = 'savli2012', desc='dasb')
madam = fetch_annotation(source='fazio2016')

p934_mni_v1 = fetch_annotation(source='gallezot2010', desc= 'p943')
p934_mni_v2 = fetch_annotation(source='savli2012', desc='p943')
#cimbi = fetch_annotation(source='beliveau2017', space='fsaverage' , desc = 'cimbi36')

In [17]:
dasb_parc = parcellater_fs164k.fit_transform(dasb, space='fsaverage', ignore_background_data=True)
dasb_mni_parc_v1 = parcellater_mni.fit_transform(dasb_mni_v1, space='mni152', ignore_background_data=True)
dasb_mni_parc_v2 = parcellater_mni.fit_transform(dasb_mni_v2, space='mni152', ignore_background_data=True)
madam_parc = parcellater_mni.fit_transform(madam, space='mni152', ignore_background_data=True)

p934_parc_v1  = parcellater_mni.fit_transform(p934_mni_v1, space='mni152', ignore_background_data=True)
p934_parc_v2  = parcellater_mni.fit_transform(p934_mni_v2, space='mni152', ignore_background_data=True)
#cimbi_parc  = parcellater_fs164k.fit_transform(cimbi, space='fsaverage', ignore_background_data=True)


In [18]:
rois = pd.read_csv(path+'parcellations/atlas-desikankilliany.csv')
rois = rois[(rois['structure'] == 'cortex')].index.to_numpy()

madam_parc = madam_parc[0][rois]
dasb_mni_parc_v1 = dasb_mni_parc_v1[0][rois]
dasb_mni_parc_v2 = dasb_mni_parc_v2[0][rois]

p934_parc_v1 = p934_parc_v1[0][rois]
p934_parc_v2 = p934_parc_v2[0][rois]

In [21]:
from scipy.stats import pearsonr
print('dasb fsaverage x madam:', pearsonr(dasb_parc, madam_parc))
print('dasb mni v1 x madam:', pearsonr(dasb_mni_parc_v1, madam_parc))
print('dasb mni v1 x v2:', pearsonr(dasb_mni_parc_v1, dasb_mni_parc_v2))

print('dasb fsaverage x dasb mni v1:', pearsonr(dasb_parc, dasb_mni_parc_v1))
print('dasb fsaverage x dasb mni v2:', pearsonr(dasb_parc, dasb_mni_parc_v2))


print('p934 v p934: ', pearsonr(p934_parc_v1, p934_parc_v2)) 

dasb fsaverage x madam: PearsonRResult(statistic=0.7220155170188297, pvalue=3.6999293936513084e-12)
dasb mni v1 x madam: PearsonRResult(statistic=0.725639606848971, pvalue=2.5597207729978666e-12)
dasb mni v1 x v2: PearsonRResult(statistic=0.9119516990230854, pvalue=3.1174408766046325e-27)
dasb fsaverage x dasb mni v1: PearsonRResult(statistic=0.9875371983856311, pvalue=9.897575707491891e-55)
dasb fsaverage x dasb mni v2: PearsonRResult(statistic=0.914563941164441, pvalue=1.203744404274862e-27)
p934 v p934:  PearsonRResult(statistic=0.8070969109453422, pvalue=9.480319384864097e-17)
